In [2]:
import pandas as pd
import numpy as np
import warnings

In [3]:
warnings.filterwarnings("ignore")

In [4]:
Base_File = 'C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Debtors GL Oct Nov/Base File for Cash flow-Oct Nov.xlsx'
Account_Master_File = 'C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Debtors GL Oct Nov/Master file for Bank accounts_v3.xlsx'
Sugar_File = 'C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Debtors GL Oct Nov/Sugar GL.xlsx'
Alcohol_File = 'C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Debtors GL Oct Nov/Alcohol GL.xlsx'
Molasses_File = 'C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Debtors GL Oct Nov/Molasses GL.xlsx'

In [5]:
# Uploading and cleaning the raw base file

df = pd.read_excel(Base_File)
df = df.iloc[6:,2:-4].reset_index()
df.columns = df.loc[[0]].values[0]
df = df.iloc[1:,1:]
df = df.dropna(how='all', axis=1)
df = df[:-4]
df.shape

(60910, 29)

In [9]:
df.columns

Index(['Assignment', 'DocumentNo', 'Doc. Date', 'Type', 'PK',
       'Amount in local cur.', 'LCurr', 'Clrng doc.', 'Clearing', 'Text',
       'Reference Key', 'Document Header Text', '       Amount in DC', 'Curr.',
       'Reference', 'Value date', 'Inv. ref.', 'D/C', 'Profit Ctr', 'AccTy',
       'Account', 'Period', 'Year', 'Pstng Date', 'CoCd', 'Reverse clearing',
       'Name of offsetting account', 'O', 'Offst.acct'],
      dtype='object')

In [10]:
# Uploading Bank Account master and merging it with the base file

Bank_Account_Master = pd.read_excel(Account_Master_File)
Bank_Account_Master = Bank_Account_Master.iloc[:,3:].reset_index()
Bank_Account_Master.columns = Bank_Account_Master.loc[[0]].values[0]
Bank_Account_Master = Bank_Account_Master.iloc[1:,1:]
Bank_Account_Master = Bank_Account_Master.dropna(how='all', axis=0)
Bank_Account_Master

,GL Account,Account Name,Type of Account,TRA/Non-TRA
1,23130561,NOD-SBI-CB-CA-34492316158[M],TRA-Principal,TRA
2,23130562,NOD-SBI-CA-316158[O],TRA-Principal,TRA
3,23130563,NOD-SBI-CA-316158[I],TRA-Principal,TRA
5,23152981,LKO-SBI-CB-CA-3537545556(M),Cane Price Payment Account,TRA
6,23152982,LKO-SBI-CB-CA-3537545556(O),Cane Price Payment Account,TRA
...,...,...,...,...
293,13100491,SBI-CMS BAJPRATAP,Collection Account,Collection Account
295,13100492,SBI-CMS RUDAULI COLLECTION ACCOUNT,Collection Account,Collection Account
297,13100493,SBI-CMS BAJUTRAULLA,Collection Account,Collection Account
299,13100494,SBI-CMS BAJKUNDARKHI,Collection Account,Collection Account


In [7]:
df = pd.merge(df, Bank_Account_Master, how = 'left', left_on = 'Account', right_on = 'GL Account')
df.shape

(60910, 33)

In [8]:
# Creating the new column

df['Deloitte Classification Head'] = np.nan

# Rule 1

### Part 1

In [9]:
df.loc[df[df.Type == 'ZR'].index,'Deloitte Classification Head'] = 'Transfer to Main account'


### Part 2

In [10]:
AB = df[df.Type == 'AB'][['DocumentNo','Amount in local cur.']]
AB = pd.DataFrame(AB.groupby('DocumentNo')['Amount in local cur.'].sum()).reset_index()


#pd.set_option('display.precision',4)
#pd.options.display.float_format = '{:.5f}'.format
NonZero = AB[(AB['Amount in local cur.'] > 0.5) | (AB['Amount in local cur.'] < -0.5)]


Zero = AB[AB['Amount in local cur.'].between(-0.5,0.5)]



In [11]:
Zero['Head'] = 'Transfer to Main account'
Zero.drop(['Amount in local cur.'],axis=1,inplace=True)
df = pd.merge(df,Zero, how='left', on = 'DocumentNo')

df['Deloitte Classification Head'].value_counts()

df.loc[df[df.Head.notnull()].index,'Deloitte Classification Head'] = 'Transfer to Main account'


In [12]:
print('--------------------Rule 1 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 1 Executed--------------------


Transfer to Main account    40169
Name: Deloitte Classification Head, dtype: int64

# Rule 2

In [13]:
df['interbank_flag'] = np.nan
filters = ['funds transf','fund transf','fund recd','funds recd','funds transfer','fund transfer','fund transfar','fund transfer', 'fund tfd', 'fund trf', 'amt trf', 'funds recd', 'fund received, trf', 'transfer entry', 'auto fund sweep']


In [14]:
df2 = df[df.Type == 'SA']

In [15]:
res = []
for i in df2['Text']:
    for j in filters:
        if j in str(i).lower():
            ind = df2[df2['Text'] == str(i)].index.tolist()
            #print(df2[df2['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [16]:
df.loc[index,'Deloitte Classification Head'] = 'Interbank transfer'

In [17]:
print('--------------------Rule 2 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 2 Executed--------------------


Transfer to Main account    40169
Interbank transfer           2386
Name: Deloitte Classification Head, dtype: int64

# Rule 3

In [18]:
DA_Cearing_doc = df[df.Type == 'DA']['DocumentNo'].values.tolist()
DZ_Cearing_doc = df[df.Type == 'DZ'][df['Clrng doc.'].notnull()]['Clrng doc.'].values.tolist()

#Check if all the doc from DA is present in clearing doc col for DZ
Reverse_Transaction_doc_DADZ = []

for i in DA_Cearing_doc:
    if i in DZ_Cearing_doc:
        Reverse_Transaction_doc_DADZ.append(i)
        
KA_Cearing_doc = df[df.Type == 'KA']['DocumentNo'].values.tolist()
KZ_Cearing_doc = df[df.Type == 'KZ'][df['Clrng doc.'].notnull()]['Clrng doc.'].values.tolist()


#Check if all the doc from KA is present in clearing doc col for KZ
Reverse_Transaction_doc_KAKZ = []

for i in KA_Cearing_doc:
    if i in KZ_Cearing_doc:
        Reverse_Transaction_doc_KAKZ.append(i)

Reverse_Transaction = pd.DataFrame(Reverse_Transaction_doc_DADZ + Reverse_Transaction_doc_KAKZ, columns = ['Clrng doc.'])
Reverse_Transaction['Reverse_Transaction_flag'] = 1

In [19]:
df = pd.merge(df,Reverse_Transaction, how='left', on = ['Clrng doc.'])

In [20]:

df.loc[df[df.Reverse_Transaction_flag.notnull()].index,'Deloitte Classification Head'] = 'Reversal transaction'


In [21]:
print('--------------------Rule 3 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 3 Executed--------------------


Transfer to Main account    40169
Interbank transfer           2386
Reversal transaction          176
Name: Deloitte Classification Head, dtype: int64


# Rule 4,5,6,7 and 8

### Collection Sugar

In [12]:
Sugar = pd.read_excel(Sugar_File)
Sugar = Sugar.iloc[6:,2:-4].reset_index()
Sugar.columns = Sugar.loc[[0]].values[0]
Sugar = Sugar.iloc[2:,1:]
Sugar = Sugar.dropna(how='all', axis=1)
Sugar = Sugar[:-3]

In [13]:
Sugar.shape

(78783, 29)

In [23]:
Sug = pd.DataFrame(Sugar['Assignment'].unique(),columns = ['Assignment_Sug'])

df = pd.merge(df, Sug, how = 'left', right_on = 'Assignment_Sug', left_on = 'Offst.acct')

index = df[df['Assignment_Sug'].notnull()][df['TRA/Non-TRA']=='Collection Account'].index
df.loc[index,'Deloitte Classification Head'] = 'Collection-Sugar'

In [24]:
print('--------------------Rule 4 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 4 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Reversal transaction          104
Name: Deloitte Classification Head, dtype: int64

### Collection Alcohol 

In [25]:
alc = pd.read_excel(Alcohol_File)
alc = alc.iloc[6:,2:-4].reset_index()#.drop(columns = 'index',inplace=True)
alc.columns = alc.loc[[0]].values[0]
alc = alc.iloc[2:,1:]
alc = alc.dropna(how='all', axis=1)
alc = alc[:-3]
alc = alc[alc.Assignment.notnull()]
alc.shape

(367, 28)

In [26]:
alcohol = pd.DataFrame(alc['Assignment'].unique(),columns = ['Assignment_alc'])

df = pd.merge(df, alcohol, how = 'left', right_on = 'Assignment_alc', left_on = 'Offst.acct')

index = df[df['Assignment_alc'].notnull()][df['TRA/Non-TRA']=='Collection Account'].index
df.loc[index,'Deloitte Classification Head'] = 'Collection-Alcohol'

In [27]:
print('--------------------Rule 5 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 5 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Reversal transaction          102
Collection-Alcohol             32
Name: Deloitte Classification Head, dtype: int64

### Collection Power

In [28]:
index = df[df['Offst.acct'] == 1011580].index
df.loc[index,'Deloitte Classification Head'] = 'Collection-Power'


In [29]:
print('--------------------Rule 6 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 6 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Reversal transaction          102
Collection-Alcohol             32
Collection-Power               17
Name: Deloitte Classification Head, dtype: int64

### Collection Molasses

In [30]:
print(Molasses_File)

C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Debtors GL Oct Nov/Molasses GL.xlsx


In [31]:
mol = pd.read_excel(Molasses_File)
mol = mol.iloc[6:,2:-4].reset_index()#.drop(columns = 'index',inplace=True)
mol.columns = mol.loc[[0]].values[0]
mol = mol.iloc[2:,1:]
mol = mol.dropna(how='all', axis=1)
mol = mol[:-3]
mol = mol[mol.Assignment.notnull()]
mol.shape

(3601, 27)

In [32]:
mol.columns

Index(['Assignment', 'DocumentNo', 'Doc. Date', 'Type', 'PK',
       ' Amt in loc.cur.', 'LCurr', 'Clrng doc.', 'Clearing', 'Text',
       'Reference Key', 'Doc.Header Text', '   Amount in DC', 'Curr.',
       'Reference', 'Inv. ref.', 'D/C', 'Profit Ctr', 'AccTy', 'Account',
       'Period', 'Year', 'Pstng Date', 'CoCd', 'Name of offsetting account',
       'O', 'Offst.acct'],
      dtype='object')

In [33]:
Molasses = pd.DataFrame(mol['Assignment'].unique(),columns = ['Assignment_mol'])
df = pd.merge(df, Molasses, how = 'left', right_on = 'Assignment_mol', left_on = 'Offst.acct')

index = df[df['Assignment_mol'].notnull()][df['TRA/Non-TRA']=='Collection Account'].index
df.loc[index,'Deloitte Classification Head'] = 'Collection-Molasses'

In [34]:
print('--------------------Rule 7 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 7 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Collection-Molasses           177
Reversal transaction           98
Collection-Alcohol             32
Collection-Power               17
Name: Deloitte Classification Head, dtype: int64

### Collection Others

In [35]:
index = df[df['TRA/Non-TRA']=='Collection Account'][df['Deloitte Classification Head'].isnull()].index
df.loc[index,'Deloitte Classification Head'] = 'Collection-Others'

In [36]:
df[df['Deloitte Classification Head'] == 'Collection-Others'].shape

(187, 40)

In [37]:
print('--------------------Rule 8 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 8 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Collection-Others             187
Collection-Molasses           177
Reversal transaction           98
Collection-Alcohol             32
Collection-Power               17
Name: Deloitte Classification Head, dtype: int64

# Rule 9 - Cane Payment

In [38]:
index = df[df['Name of offsetting account'] == 'Cane Liability'][df['Amount in local cur.']<0].index
df.loc[index,'Deloitte Classification Head'] = 'Cane payment'


In [39]:
print('--------------------Rule 9 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 9 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Cane payment                  536
Collection-Others             187
Collection-Molasses           177
Reversal transaction           98
Collection-Alcohol             32
Collection-Power               17
Name: Deloitte Classification Head, dtype: int64

# Rule 10 - Cane payment-return

In [40]:
index = df[df['Name of offsetting account'] == 'Cane Liability'][df['Amount in local cur.']>0].index
df.loc[index,'Deloitte Classification Head'] = 'Cane payment-return'

In [41]:
print('--------------------Rule 10 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 10 Executed--------------------


Transfer to Main account    40169
Collection-Sugar             8298
Interbank transfer           2386
Cane payment                  536
Collection-Others             187
Collection-Molasses           177
Reversal transaction           98
Collection-Alcohol             32
Collection-Power               17
Cane payment-return            15
Name: Deloitte Classification Head, dtype: int64

# Rule 11 

In [42]:
#finding the index of all offst acnt beginning with 400
ind = []
for i in range(len(df['Offst.acct'])):
    if str(df['Offst.acct'][i])[:3] == '400':
        ind.append(i)
#finding the index of all offst acnt beginning with 400 and having text = 'can tpt' or 'cane transportation'

df11 = df.loc[ind]

In [43]:
res = []
for i in df11['Text']:
    if 'cane tpt' in str(i).lower():
        ind = df11[df11['Text'] == str(i)].index.tolist()
        #print(df11[df11['Text'] == str(i)].index)
        res.append(ind)
    
    elif 'cane transportation' in str(i).lower():
        ind = df11[df11['Text'] == str(i)].index.tolist()
        #print(df11[df11['Text'] == str(i)].index)
        res.append(ind)    

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [44]:
df.loc[index,'Deloitte Classification Head'] = 'Cane transportation expenses'

In [45]:
print('--------------------Rule 11 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 11 Executed--------------------


Transfer to Main account        40169
Collection-Sugar                 8298
Interbank transfer               2386
Cane payment                      536
Collection-Others                 187
Collection-Molasses               177
Reversal transaction               98
Cane transportation expenses       35
Collection-Alcohol                 32
Collection-Power                   17
Cane payment-return                15
Name: Deloitte Classification Head, dtype: int64

# Rule 12

In [46]:
res = []
for i in df['Text']:
      if 'cane loading' in str(i).lower():
            ind = df[df['Text'] == str(i)].index.tolist()
            #print(df[df['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [47]:
df.loc[index,'Deloitte Classification Head'] = 'Cane loading expenses'

In [48]:
print('--------------------Rule 12 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 12 Executed--------------------


Transfer to Main account        40169
Collection-Sugar                 8298
Interbank transfer               2386
Cane payment                      536
Collection-Others                 187
Collection-Molasses               177
Reversal transaction               98
Cane transportation expenses       35
Collection-Alcohol                 32
Cane loading expenses              21
Collection-Power                   17
Cane payment-return                15
Name: Deloitte Classification Head, dtype: int64

# Rule 13

In [49]:
res = []
for i in df['Text']:
      if 'cane comm' in str(i).lower():
            ind = df[df['Text'] == str(i)].index.tolist()
            #print(df[df['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [50]:
df.loc[index,'Deloitte Classification Head'] = 'Cane commission'

In [51]:
print('--------------------Rule 13 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 13 Executed--------------------


Transfer to Main account        40169
Collection-Sugar                 8298
Interbank transfer               2386
Cane payment                      536
Collection-Others                 187
Collection-Molasses               177
Reversal transaction               94
Cane commission                    93
Cane transportation expenses       35
Collection-Alcohol                 32
Cane loading expenses              21
Collection-Power                   17
Cane payment-return                15
Name: Deloitte Classification Head, dtype: int64

# Rule 14

In [52]:
index = df[df['Name of offsetting account'] == 'Railway Freight Advance Account'].index

In [53]:
df.loc[index,'Deloitte Classification Head'] = 'Sugar transportation expenses'

In [54]:
print('--------------------Rule 14 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 14 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2386
Cane payment                       536
Collection-Others                  187
Collection-Molasses                177
Reversal transaction                94
Cane commission                     93
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Name: Deloitte Classification Head, dtype: int64

# Rule 15

In [55]:
res = []
for i in df['Text']:
    if 'sugar handling' in str(i).lower():
        ind = df[df['Text'] == str(i)].index.tolist()
        #print(df11[df11['Text'] == str(i)].index)
        res.append(ind)
    
    elif 'sugar loading and unloading' in str(i).lower():
        ind = df[df['Text'] == str(i)].index.tolist()
        #print(df11[df11['Text'] == str(i)].index)
        res.append(ind)    

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [56]:
index

[51429, 51751, 54277, 54568, 55281, 55361, 57165]

In [57]:
df.loc[index,'Deloitte Classification Head'] = 'Sugar handling charges'

In [58]:
print('--------------------Rule 15 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 15 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2386
Cane payment                       536
Collection-Others                  187
Collection-Molasses                177
Reversal transaction                94
Cane commission                     93
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Name: Deloitte Classification Head, dtype: int64

# Rule 16

In [59]:
res = []
for i in df['Text']:
      if 'payment against commission' in str(i).lower():
            ind = df[df['Text'] == str(i)].index.tolist()
            #print(df[df['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [60]:
df.loc[index,'Deloitte Classification Head'] = 'Sugar selling commission'

In [61]:
print('--------------------Rule 16 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 16 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2386
Cane payment                       536
Collection-Others                  187
Collection-Molasses                177
Sugar selling commission           121
Reversal transaction                94
Cane commission                     93
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Name: Deloitte Classification Head, dtype: int64

# Rule 17

In [62]:
index = df[df['Name of offsetting account'] == 'Salary Payable A/c'].index

In [63]:
df.loc[index,'Deloitte Classification Head'] = 'Salary payments'

In [64]:
print('--------------------Rule 17 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 17 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2386
Cane payment                       536
Collection-Others                  187
Collection-Molasses                177
Sugar selling commission           121
Reversal transaction                94
Cane commission                     93
Salary payments                     56
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Name: Deloitte Classification Head, dtype: int64

# Rule 18

In [65]:
index = df[df['Name of offsetting account'] == 'Duty Draw Back Receivable'].index

In [66]:
df.loc[index,'Deloitte Classification Head'] = 'Refund of stat dues'

In [67]:
print('--------------------Rule 18 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 18 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2386
Cane payment                       536
Collection-Others                  182
Collection-Molasses                177
Sugar selling commission           121
Reversal transaction                94
Cane commission                     93
Salary payments                     56
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Name: Deloitte Classification Head, dtype: int64

# Rule 19

In [68]:
index = df[df['Name of offsetting account'] == 'Bank Charges'].index

In [69]:
df.loc[index,'Deloitte Classification Head'] = 'Bank Charges'

In [70]:
print('--------------------Rule 19 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 19 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       536
Collection-Molasses                177
Collection-Others                  176
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Salary payments                     56
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Name: Deloitte Classification Head, dtype: int64

# Rule 21

In [71]:
index = df[df['Type of Account'] == 'TRA-Capital Expenditure'][df.Type != 'ZR'][df['Deloitte Classification Head'].isnull()].index

In [72]:
df.loc[index,'Deloitte Classification Head'] = 'Capital expenditure'

In [73]:
print('--------------------Rule 21 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 21 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       536
Collection-Molasses                177
Collection-Others                  176
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Capital expenditure                 58
Salary payments                     56
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Name: Deloitte Classification Head, dtype: int64

# Rule 22

In [74]:
df22 = df[df['Type of Account'] == 'Cane Price Payment Account']

In [75]:
res = []
for i in df22['Text']:
      if 'advance' in str(i).lower():
            ind = df22[df22['Text'] == str(i)].index.tolist()
            #print(df22[df22['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [76]:
len(index)

140

In [77]:
df.loc[index,'Deloitte Classification Head'] = 'Loan to farmers'

In [78]:
print('--------------------Rule 22 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 22 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Capital expenditure                 58
Salary payments                     56
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Name: Deloitte Classification Head, dtype: int64

# Rule 23

In [79]:
index = df[df['Name of offsetting account'] == 'Cane Seed Loan Account'].index

In [80]:
df.loc[index,'Deloitte Classification Head'] = 'Advance for cane seed'

In [81]:
print('--------------------Rule 23 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 23 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Capital expenditure                 58
Salary payments                     56
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Advance for cane seed                2
Name: Deloitte Classification Head, dtype: int64

# Rule 24 

In [82]:
index = df[df['Type of Account'] == 'TRA-Debt Service'][df['Name of offsetting account'] == 'Interest Accrued and Due'].index

In [83]:
df.loc[index,'Deloitte Classification Head'] = 'Interest payment'

In [84]:
print('--------------------Rule 24 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 24 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Capital expenditure                 58
Salary payments                     56
Interest payment                    55
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Advance for cane seed                2
Name: Deloitte Classification Head, dtype: int64

# Rule 25

In [85]:
df25 = df[df['Type of Account'] == 'TRA-Debt Service']

In [86]:
res = []
for i in df25['Text']:
    if 'instalment' in str(i).lower():
        ind = df25[df25['Text'] == str(i)].index.tolist()
        #print(df25[df25['Text'] == str(i)].index)
        res.append(ind)
    elif 'repayment' in str(i).lower():
        ind = df25[df25['Text'] == str(i)].index.tolist()
        #print(df25[df25['Text'] == str(i)].index)
        res.append(ind)
        
    
index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [87]:
df.loc[index,'Deloitte Classification Head'] = 'Repayment of Term loan'

In [88]:
print('--------------------Rule 25 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 25 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Capital expenditure                 58
Salary payments                     56
Interest payment                    55
Repayment of Term loan              38
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Advance for cane seed                2
Name: Deloitte Classification Head, dtype: int64

# Rule 26

In [89]:
res = []
for i in df['Text']:
      if 'cash deposit' in str(i).lower():
            ind = df[df['Text'] == str(i)].index.tolist()
            #print(df[df['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [90]:
df.loc[index,'Deloitte Classification Head'] = 'Cash deposit'

In [91]:
print('--------------------Rule 26 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 26 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Capital expenditure                 58
Salary payments                     56
Interest payment                    55
Repayment of Term loan              38
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Cash deposit                         4
Advance for cane seed                2
Name: Deloitte Classifica

# Rule 27

In [92]:
res = []
for i in df['Text']:
      if 'cash withdrawl' in str(i).lower():
            ind = df[df['Text'] == str(i)].index.tolist()
            #print(df[df['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [93]:
df.loc[index,'Deloitte Classification Head'] = 'Cash withdrawal'

In [94]:
print('--------------------Rule 27 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 27 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Cash withdrawal                     67
Capital expenditure                 58
Salary payments                     56
Interest payment                    55
Repayment of Term loan              38
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Sugar handling charges               7
Refund of stat dues                  5
Cash deposit                         4
Advance for cane seed    

# Rule 28

In [95]:
res = []
for i in df['Text']:
      if 'cane center' in str(i).lower():
            ind = df[df['Text'] == str(i)].index.tolist()
            #print(df[df['Text'] == str(i)].index)
            res.append(ind)

index = []
for i in res:
    for j in i:
        if j not in index:
            index.append(j)

In [96]:
df.loc[index,'Deloitte Classification Head'] = 'Cane centre expenses'

In [97]:
print('--------------------Rule 28 Executed--------------------')
df['Deloitte Classification Head'].value_counts()

--------------------Rule 28 Executed--------------------


Transfer to Main account         40169
Collection-Sugar                  8298
Interbank transfer                2383
Cane payment                       529
Collection-Molasses                177
Collection-Others                  176
Loan to farmers                    140
Sugar selling commission           121
Bank Charges                       101
Reversal transaction                94
Cane commission                     93
Cash withdrawal                     67
Capital expenditure                 58
Salary payments                     56
Interest payment                    55
Repayment of Term loan              38
Sugar transportation expenses       36
Cane transportation expenses        35
Collection-Alcohol                  32
Cane loading expenses               21
Collection-Power                    17
Cane payment-return                 15
Cane centre expenses                14
Sugar handling charges               7
Refund of stat dues                  5
Cash deposit             

In [98]:
final = df.iloc[:,:-6]

In [99]:
pf = pd.read_excel('C:/Users/skesh/Desktop/Cash flow automation/Oct Nov/Output_Praveen.xlsx')
Output = pd.merge(final,pf[['Deloitte Comments']],how='left',left_on = final.index, right_on = pf[['Deloitte Comments']].index)
Output = Output.iloc[:,1:]

In [100]:
#Output.to_excel('C:/Users/skesh/Desktop/Screenshots/Jobs/Cash Flow Automation.xlsx')

In [101]:
#val = Output[Output['Deloitte Classification Head'] != Output['Deloitte Comments']][Output['Deloitte Classification Head'].notnull()]

In [102]:
#val.to_excel('Cash Flow Automation Validation.xlsx')

In [103]:
pf['Deloitte Comments'].value_counts().head(12)

Transfer to Main account                40179
Collection-Sugar                         8272
Interbank transfer                       2533
Purchase of stores & spares              1298
Cane payment                             1207
Repairs & maintenance                    1124
Employee advances and reimbursements      840
Admin expenses                            577
Contractor payments                       528
Loan to farmers                           519
Reversal transaction                      394
Cane payment-return                       331
Name: Deloitte Comments, dtype: int64